In [1]:
%load_ext autoreload
%autoreload 2

# Import and Setup

In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import gc
import json
import string
import random
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from tqdm import tqdm
from functools import partial
from argparse import Namespace
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

import wandb
from wandb.keras import WandbCallback

from model import SimpleSupervisedtModel, ArcFaceSupervisedModel, get_feature_extractor
from config import get_test_config
from data import GetDataloader
from utils import ShowBatch
from callbacks import GetCallbacks

In [10]:
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [11]:
args = get_test_config()
args.exp_id = '23005S4V'

DEBUG = True
MODEL_PATH = f'{args.model_save_path}/{args.exp_id}'
TRAIN_EMBED_PATH = f'{args.embedding_save_path}/{args.exp_id}'

# Load Dataframes

In [12]:
train_df = pd.read_csv('../cleaned_train_5_fold_individual_ids.csv')

with open('../label2ids_individual_ids.json') as json_file:
    label2ids = json.load(json_file)
    
args.num_labels = len(label2ids)
args.labels = label2ids

train_df.head()

,image,species,individual_id,img_path,fold,target
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,../128x128/train_images-128-128/train_images-1...,0,0
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,../128x128/train_images-128-128/train_images-1...,2,1
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,../128x128/train_images-128-128/train_images-1...,4,2
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,../128x128/train_images-128-128/train_images-1...,4,3
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,../128x128/train_images-128-128/train_images-1...,1,4


In [13]:
def source_path(row):
    return f'{args.test_img_path}/{row.image}'

test_df = pd.read_csv('../sample_submission.csv')
test_df['img_path'] = test_df.apply(lambda row: source_path(row), axis=1)

test_df.head()

,image,predictions,img_path
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,../128x128/test_images-128-128/test_images-128...
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,../128x128/test_images-128-128/test_images-128...
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,../128x128/test_images-128-128/test_images-128...
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,../128x128/test_images-128-128/test_images-128...
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...,../128x128/test_images-128-128/test_images-128...


# Get Train Embeddings

In [37]:
train_embeddings = []
train_index = []

for fold in tqdm(range(args.num_folds)):
    data = np.load(f'{TRAIN_EMBED_PATH}/embedding_{fold}.npz', allow_pickle=True)
    train_embeddings.extend(data['embedding'])
    train_index.extend(data['index'])
    
train_embeddings = np.array(train_embeddings)
train_index = np.array(train_index)

print(train_embeddings.shape, train_index.shape)

100% 5/5 [00:00<00:00, 12.81it/s]

(51033, 1280) (51033,)


In [ ]:
MODEL_PATH = f'{args.model_save_path}/{args.exp_id}'
train_preds = []

for fold in tqdm(range(args.num_folds)):
    valid_df = train_df[train_df.fold == fold]
    dataset = GetDataloader(args)
    validloader = dataset.dataloader(valid_df, data_type='valid')
    
    model = tf.keras.models.load_model(f'{MODEL_PATH}/model_{fold}')
    
    preds = model.predict(validloader)
    train_preds.extend(np.argmax(preds, axis=1))
    
train_preds = np.array(train_preds)

In [36]:
train_index

array([    0,     1,     2, ..., 51030, 51031, 51032])

In [30]:
inds = train_index.argsort()

train_embeddings = train_embeddings[inds]
train_preds = train_preds[inds]
train_index = train_index[inds]

In [31]:
train_df['preds'] = train_preds

In [35]:
train_df

,image,species,individual_id,img_path,fold,target,preds
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,../128x128/train_images-128-128/train_images-1...,0,0,9439
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,../128x128/train_images-128-128/train_images-1...,2,1,14673
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,../128x128/train_images-128-128/train_images-1...,4,2,146
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,../128x128/train_images-128-128/train_images-1...,4,3,587
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,../128x128/train_images-128-128/train_images-1...,1,4,437
...,...,...,...,...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5ac053677ed1,../128x128/train_images-128-128/train_images-1...,1,96,1952
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3,../128x128/train_images-128-128/train_images-1...,0,1482,1167
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9,../128x128/train_images-128-128/train_images-1...,0,8220,9840
51031,fffbc5dd642d8c.jpg,beluga,4000b3d7c24e,../128x128/train_images-128-128/train_images-1...,4,13202,528


In [13]:
# MODEL_PATH = f'{args.model_save_path}/{args.exp_id}'
# train_embeddings = []

# for fold in tqdm(range(args.num_folds)):
#     valid_df = train_df[train_df.fold == fold]
#     dataset = GetDataloader(args)
#     validloader = dataset.dataloader(valid_df, data_type='valid')
    
#     model = tf.keras.models.load_model(f'{MODEL_PATH}/model_{fold}.h5')
#     feature_extractor = get_feature_extractor(model)

#     embeddings = feature_extractor.predict(validloader)
#     train_embeddings.extend(embeddings)
    
# train_embeddings = np.array(train_embeddings)

100% 5/5 [00:34<00:00,  6.95s/it]


# Learn Nearest Neighbors - Unsupervised

In [52]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=6, metric='cosine')
neigh.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=6)

# Get Test Embeddings

In [18]:
dataset = GetDataloader(args)
testloader = dataset.dataloader(test_df, data_type='test')

In [19]:
test_embeddings = []

for fold in tqdm(range(args.num_folds)):
    model = tf.keras.models.load_model(f'{MODEL_PATH}/model_{fold}')
    feature_extractor = get_feature_extractor(model)

    embeddings = feature_extractor.predict(testloader)

    test_embeddings.append(embeddings)
    
test_embeddings = np.array(test_embeddings)
test_embeddings = np.mean(test_embeddings, axis=0)
print(test_embeddings.shape)

100% 5/5 [02:00<00:00, 24.03s/it]


(27956, 1280)


# Find the Nearest Ids

In [68]:
sub_df = test_df[['image', 'predictions']]

In [70]:
dist, idxs = neigh.kneighbors(test_embeddings, n_neighbors=6)

In [71]:
for i in tqdm(range(len(sub_df))):
    individual_ids = train_df.loc[idxs[i].flatten().tolist()[:5]].individual_id.values
    individual_ids = ' '.join(individual_ids)
    sub_df.loc[i, 'predictions'] = individual_ids

100% 27956/27956 [00:14<00:00, 1969.97it/s]


In [72]:
SUBMISSION_DIR = f'../submissions/{args.exp_id}'
os.makedirs(SUBMISSION_DIR, exist_ok=True)
sub_df.to_csv(f'{SUBMISSION_DIR}/submission.csv', index=False)

# Log LB Score

In [29]:
run = wandb.init(project='happywhale',
                 config=vars(args),
                 group=f'effnetb0',
                 job_type='inference',
                 name=f'{args.exp_id}-infer')

wandb.log({'LB Score': 0.038})
wandb.finish()

LB Score,▁
LB Score,0.038
